# Francy

### A list of all Francy features, with examples, is available on the following notebook [Francy Features](francy-features.ipynb).
### Francy can be used to create new interactive packages, for instance the [FrancyMonoids](francy-monoids.ipynb) or [SubgroupLattice](subgroup-lattice.ipynb) packages.
### Francy has been extended and supports, apart from GAP, [Python](https://github.com/zerline/francy-widget) and [Julia](https://github.com/fieker/Francy) languages as backends.
### Francy inspired projects such as [JupyterViz](https://github.com/nathancarter/jupyterviz).

# How to make an existing Package interactive? 
### Francy can be used to enhance existing packages by providing interactive visualizations.
### As an example, lets use the package [Digraphs](https://www.gap-system.org/Packages/digraphs.html) from James Mitchell. This package already allows the creation of static images using GraphViz.
### Lets start by loading GAP packages for the demo:

In [2]:
LoadPackage("francy");;
LoadPackage("digraph");;

 ### Imagine we want to represent the directed graph of all subgroups of the DihedralGroup $D_4$:

In [5]:
G            := DihedralGroup(4);
allSubgroups := AllSubgroups(G);
digraph      := Digraph(allSubgroups, {H, K} -> IsSubgroup(H, K));

<group of size 4 with 2 generators>

[ <trivial group>, <group of size 2 with 1 generators>, <group of size 2 with 1 generators>, <group of size 2 with 1 generators>, <group of size 4 with 2 generators> ]

<digraph with 5 vertices, 12 edges>

### Display the Digraph:

In [6]:
dot := DotDigraph(digraph);
JupyterSplashDot(dot);

Error, operations: IN of character and boolean or fail is not defined in
  if '/' in path then
if not IsExecutableFile( path ) then
return fail;
else
return path;
fi;
else
path := Filename( DirectoriesSystemPrograms(  ), path );
if path = fail then
return fail;
fi;
if not IsExecutableFile( path ) then
return fail;
fi;
return path;
fi; at /home/ec2-user/environment/gap/pkg/io-4.6.0/gap/io.gi:956 called from 
IO_FindExecutable( path ) at /home/ec2-user/environment/gap/pkg/io-4.6.0/gap/io.gi:1047 called from
IO_Popen( IO_FindExecutable( "dot" ), [ "-Tsvg", fn ], "r" ) at /home/ec2-user/environment/gap/pkg/JupyterKernel-1.3/gap/JupyterUtil.gi:16 called from
<function "JupyterSplashDot">( <arguments> )
 called from read-eval loop at stream:2


"//dotdigraph hgn{node [shape=circle]123451 -> 12 -> 12 -> 23 -> 13 -> 34 -> 14 -> 45 -> 15 -> 25 -> 35 -> 45 -> 5}"

### Now, we can obtain the same representation using Francy by:

In [15]:
nodes    := [];;
vertices := DigraphVertices(digraph);;
edges    := DigraphEdges(digraph);;

graph := Graph(GraphType.DIRECTED);;

for i in vertices do
    nodes[i] := Shape(ShapeType.CIRCLE, String(i));;
    Add(graph, nodes[i]);;
od;;

for i in edges do
    Add(graph, Link(nodes[i[1]], nodes[i[2]]));;
od;;

canvas := Canvas(Concatenation("Subgroups Digraph of ", String(G)));;
Add(canvas, graph);;

Draw(canvas);

### The default renderer is D3, so in order to view the Graphviz representation, Just switch the renderer  by going to 'Settings -> Renderers' and select Graphviz-Renderer.
### Now, if we wanted to make it interactive by checking whether a subgroup is simple or not, for instance, we could extend the previous code:

In [16]:
FrancyDigraphs := function(G)
    local as, d, v, e, graph, nodes, m, IsGroupSimple, canvas, i;

    as := AllSubgroups(G);

    d := Digraph(as, {H, K} -> IsSubgroup(H, K));;
    v := DigraphVertices(d);;
    e := DigraphEdges(d);;

    graph := Graph(GraphType.DIRECTED);;
    canvas := Canvas(Concatenation("Subgroups Digraph of ", String(G)));;
    Add(canvas, graph);;

    nodes := [];;

    # Explanatory message to show everytime
    m := FrancyMessage(FrancyMessageType.INFO, "Simple Groups", 
    "A group is simple if it is nontrivial and has no nontrivial normal subgroups.");;
    
    # Add a new function to be invoked on click
    IsGroupSimple := function(i)
        Add(canvas, m);;
        if IsSimpleGroup(as[i]) then
            Add(canvas, FrancyMessage("Simple", Concatenation("The vertex ", 
                String(i), ", representing the subgroup ", String(as[i]), ", is simple.")));;
        else
            Add(canvas, FrancyMessage("Not Simple", Concatenation("The vertex ", 
                String(i), ", representing the subgroup ", String(as[i]), ", is not simple.")));;
        fi;;
        return Draw(canvas);
    end;;

    for i in v do
        nodes[i] := Shape(ShapeType.CIRCLE, String(i));;
        # Add a menu on each node with a callback to the function created before
        Add(nodes[i], Menu("Simple", Callback(IsGroupSimple, [i])));;
        Add(graph, nodes[i]);;
    od;;

    for i in e do
        Add(graph, Link(nodes[i[1]], nodes[i[2]]));;
    od;;

    return Draw(canvas);
end;

function( G ) ... end

In [17]:
FrancyDigraphs(G);